# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import pickle


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
path = r'D:\Documentos\DataScience_Udacity\4. Data Engineering\5. Final Project\Projeto\data\DisasterResponseData.db'

engine = create_engine('sqlite:///' + path)
df = pd.read_sql_table(table_name= 'T_DisasterResponse', con= engine)

X = df['message']
Y = df.drop(columns= ['original', 'id', 'message', 'genre'])

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, Y, test_size=0.33, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002456622B940>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)

print(classification_report(y_test.values, y_pred, target_names= Y.columns, zero_division= False))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.89      6566
               request       0.88      0.44      0.59      1466
                 offer       0.00      0.00      0.00        46
           aid_related       0.79      0.61      0.69      3541
          medical_help       0.67      0.06      0.11       662
      medical_products       0.81      0.06      0.11       426
     search_and_rescue       0.70      0.06      0.11       235
              security       1.00      0.01      0.01       159
              military       0.82      0.03      0.06       287
           child_alone       0.00      0.00      0.00         0
                 water       0.92      0.26      0.40       551
                  food       0.87      0.43      0.58       962
               shelter       0.90      0.20      0.32       731
              clothing       0.83      0.04      0.08       123
                 money       0.75      

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [8]:
parameters = {
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_leaf': [5, 10]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, verbose= 2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed:  2.8min remaining:  4.2min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  4.2min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002456622B940>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'clf__estimator__min_samples_leaf': [5, 10],
                         'clf__estimator__n_estimators': [50, 100]},
             verbose=2)

In [10]:
best_model = cv.best_params_
best_model

{'clf__estimator__min_samples_leaf': 5, 'clf__estimator__n_estimators': 50}

In [11]:
y_pred_improved = cv.predict(X_test)

In [28]:
print(classification_report(y_test.values, y_pred_improved, target_names= Y.columns, zero_division= False))

                        precision    recall  f1-score   support

               related       0.78      0.99      0.87      6566
               request       0.94      0.14      0.25      1466
                 offer       0.00      0.00      0.00        46
           aid_related       0.78      0.59      0.67      3541
          medical_help       0.60      0.00      0.01       662
      medical_products       1.00      0.00      0.01       426
     search_and_rescue       0.00      0.00      0.00       235
              security       0.00      0.00      0.00       159
              military       0.00      0.00      0.00       287
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       551
                  food       0.81      0.02      0.03       962
               shelter       0.88      0.01      0.02       731
              clothing       0.00      0.00      0.00       123
                 money       0.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [25]:
sys.path

['d:\\Documentos\\DataScience_Udacity\\4. Data Engineering\\5. Final Project\\Projeto\\jupyter',
 'c:\\Users\\lucas\\.vscode\\extensions\\ms-toolsai.jupyter-2021.8.1054968649\\pythonFiles',
 'c:\\Users\\lucas\\.vscode\\extensions\\ms-toolsai.jupyter-2021.8.1054968649\\pythonFiles\\lib\\python',
 'D:\\Programas\\Anaconda3\\python38.zip',
 'D:\\Programas\\Anaconda3\\DLLs',
 'D:\\Programas\\Anaconda3\\lib',
 'D:\\Programas\\Anaconda3',
 '',
 'D:\\Programas\\Anaconda3\\lib\\site-packages',
 'D:\\Programas\\Anaconda3\\lib\\site-packages\\win32',
 'D:\\Programas\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'D:\\Programas\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'D:\\Programas\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\lucas\\.ipython']

In [27]:
model_file = r'D:\Documentos\DataScience_Udacity\4. Data Engineering\5. Final Project\Projeto\model\model.pkl'

with open(model_file, 'wb') as model_file:
  pickle.dump(pipeline, model_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.